In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats
import sys

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
#offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
#inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
#full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [4]:
assortment_info_df_20 = assortment_info_df[(assortment_info_df['offer_times']>=20)]
assortments_20 = []
for i in range(len(assortment_info_df_20['assortments'])):
    assortments_20.append(assortment_info_df_20['assortments'][i])


jd_choice_20 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_20:
        jd_choice_20.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_20 = assortment_info_df_20['assortments']
choice_collection_20 = np.zeros((n,len(collection_20)))

frequency_collection_20 = np.zeros((n,len(collection_20)))

purchaseset_list_20 = jd_choice_20['purchaseset'].tolist()

inc_prod_num_20 = assortment_info_df_20['includ_prod_num']
choice_prob_20 = jd_choice_20['choice prob']
frequency_20 = jd_choice_20['purchasenum']

global_num = 0
for i in range(len(collection_20)):
    local_num =0
    while(local_num < inc_prod_num_20[i]):
        if purchaseset_list_20[global_num][0] in assortments_20[i]:
            choice_collection_20[purchaseset_list_20[global_num][0]][i] = choice_prob_20[global_num]
            frequency_collection_20[purchaseset_list_20[global_num][0]][i] = frequency_20[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_20[global_num])
            print(assortments_20[i])
            print('error!')
            break
#print(global_num)  

choice_check_20 = pd.DataFrame(choice_collection_20)
#choice_check_20.to_csv('instances/full_instances/full_20.csv')
frequency_collection_20 = frequency_collection_20.astype(int)
frequency_check_20 = pd.DataFrame(frequency_collection_20)
#frequency_check_20.to_csv('instances/full_instances/frequency_20.csv')

offer_times_20 = assortment_info_df_20['offer_times'].tolist()
print('the number of assortments with at least 20 purchases is',len(assortments_20))

the number of assortments with at least 20 purchases is 29


In [5]:
assortment_info_df_30 = assortment_info_df[(assortment_info_df['offer_times']>=30)]
assortments_30 = []
for i in range(len(assortment_info_df_30['assortments'])):
    assortments_30.append(assortment_info_df_30['assortments'][i])


jd_choice_30 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_30:
        jd_choice_30.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_30 = assortment_info_df_30['assortments']
choice_collection_30 = np.zeros((n,len(collection_30)))

frequency_collection_30 = np.zeros((n,len(collection_30)))

purchaseset_list_30 = jd_choice_30['purchaseset'].tolist()

inc_prod_num_30 = assortment_info_df_30['includ_prod_num']
choice_prob_30 = jd_choice_30['choice prob']
frequency_30 = jd_choice_30['purchasenum']

global_num = 0
for i in range(len(collection_30)):
    local_num =0
    while(local_num < inc_prod_num_30[i]):
        if purchaseset_list_30[global_num][0] in assortments_30[i]:
            choice_collection_30[purchaseset_list_30[global_num][0]][i] = choice_prob_30[global_num]
            frequency_collection_30[purchaseset_list_30[global_num][0]][i] = frequency_30[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_30[global_num])
            print(assortments_30[i])
            print('error!')
            break
#print(global_num)  

choice_check_30 = pd.DataFrame(choice_collection_30)
#choice_check_30.to_csv('instances/full_instances/full_30.csv')
frequency_collection_30 = frequency_collection_30.astype(int)
frequency_check_30 = pd.DataFrame(frequency_collection_30)
#frequency_check_30.to_csv('instances/full_instances/frequency_30.csv')

offer_times_30 = assortment_info_df_30['offer_times'].tolist()
print('the number of assortments with at least 30 purchases is',len(assortments_30))

the number of assortments with at least 30 purchases is 24


In [6]:
assortment_info_df_40 = assortment_info_df[(assortment_info_df['offer_times']>=40)]
assortments_40 = []
for i in range(len(assortment_info_df_40['assortments'])):
    assortments_40.append(assortment_info_df_40['assortments'][i])


jd_choice_40 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_40:
        jd_choice_40.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_40 = assortment_info_df_40['assortments']
choice_collection_40 = np.zeros((n,len(collection_40)))

frequency_collection_40 = np.zeros((n,len(collection_40)))

purchaseset_list_40 = jd_choice_40['purchaseset'].tolist()

inc_prod_num_40 = assortment_info_df_40['includ_prod_num']
choice_prob_40 = jd_choice_40['choice prob']
frequency_40 = jd_choice_40['purchasenum']

global_num = 0
for i in range(len(collection_40)):
    local_num =0
    while(local_num < inc_prod_num_40[i]):
        if purchaseset_list_40[global_num][0] in assortments_40[i]:
            choice_collection_40[purchaseset_list_40[global_num][0]][i] = choice_prob_40[global_num]
            frequency_collection_40[purchaseset_list_40[global_num][0]][i] = frequency_40[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_40[global_num])
            print(assortments_40[i])
            print('error!')
            break
#print(global_num)  

choice_check_40 = pd.DataFrame(choice_collection_40)
#choice_check_40.to_csv('instances/full_instances/full_40.csv')
frequency_collection_40 = frequency_collection_40.astype(int)
frequency_check_40 = pd.DataFrame(frequency_collection_40)
#frequency_check_40.to_csv('instances/full_instances/frequency_40.csv')

offer_times_40 = assortment_info_df_40['offer_times'].tolist()
print('the number of assortments with at least 40 purchases is',len(assortments_40))

the number of assortments with at least 40 purchases is 19


In [7]:
assortment_info_df_50 = assortment_info_df[(assortment_info_df['offer_times']>=50)]
assortments_50 = []
for i in range(len(assortment_info_df_50['assortments'])):
    assortments_50.append(assortment_info_df_50['assortments'][i])


jd_choice_50 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_50:
        jd_choice_50.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_50 = assortment_info_df_50['assortments']
choice_collection_50 = np.zeros((n,len(collection_50)))

frequency_collection_50 = np.zeros((n,len(collection_50)))

purchaseset_list_50 = jd_choice_50['purchaseset'].tolist()

inc_prod_num_50 = assortment_info_df_50['includ_prod_num']
choice_prob_50 = jd_choice_50['choice prob']
frequency_50 = jd_choice_50['purchasenum']

global_num = 0
for i in range(len(collection_50)):
    local_num =0
    while(local_num < inc_prod_num_50[i]):
        if purchaseset_list_50[global_num][0] in assortments_50[i]:
            choice_collection_50[purchaseset_list_50[global_num][0]][i] = choice_prob_50[global_num]
            frequency_collection_50[purchaseset_list_50[global_num][0]][i] = frequency_50[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_50[global_num])
            print(assortments_50[i])
            print('error!')
            break
#print(global_num)  

choice_check_50 = pd.DataFrame(choice_collection_50)
#choice_check_50.to_csv('instances/full_instances/full_50.csv')
frequency_collection_50 = frequency_collection_50.astype(int)
frequency_check_50 = pd.DataFrame(frequency_collection_50)
#frequency_check_50.to_csv('instances/full_instances/frequency_50.csv')

offer_times_50 = assortment_info_df_50['offer_times'].tolist()
print('the number of assortments with at least 50 purchases is',len(assortments_50))

the number of assortments with at least 50 purchases is 15


In [8]:
assortment_info_df_60 = assortment_info_df[(assortment_info_df['offer_times']>=60)]
assortments_60 = []
for i in range(len(assortment_info_df_60['assortments'])):
    assortments_60.append(assortment_info_df_60['assortments'][i])


jd_choice_60 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_60:
        jd_choice_60.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_60 = assortment_info_df_60['assortments']
choice_collection_60 = np.zeros((n,len(collection_60)))

frequency_collection_60 = np.zeros((n,len(collection_60)))

purchaseset_list_60 = jd_choice_60['purchaseset'].tolist()

inc_prod_num_60 = assortment_info_df_60['includ_prod_num']
choice_prob_60 = jd_choice_60['choice prob']
frequency_60 = jd_choice_60['purchasenum']

global_num = 0
for i in range(len(collection_60)):
    local_num =0
    while(local_num < inc_prod_num_60[i]):
        if purchaseset_list_60[global_num][0] in assortments_60[i]:
            choice_collection_60[purchaseset_list_60[global_num][0]][i] = choice_prob_60[global_num]
            frequency_collection_60[purchaseset_list_60[global_num][0]][i] = frequency_60[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_60[global_num])
            print(assortments_60[i])
            print('error!')
            break
#print(global_num)  

choice_check_60 = pd.DataFrame(choice_collection_60)
#choice_check_60.to_csv('instances/full_instances/full_60.csv')
frequency_collection_60 = frequency_collection_60.astype(int)
frequency_check_60 = pd.DataFrame(frequency_collection_60)
#frequency_check_60.to_csv('instances/full_instances/frequency_60.csv')

offer_times_60 = assortment_info_df_60['offer_times'].tolist()
print('the number of assortments with at least 60 purchases is',len(assortments_60))

the number of assortments with at least 60 purchases is 13


In [9]:
assortment_info_df_75 = assortment_info_df[(assortment_info_df['offer_times']>=75)]
assortments_75 = []
for i in range(len(assortment_info_df_75['assortments'])):
    assortments_75.append(assortment_info_df_75['assortments'][i])


jd_choice_75 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_75:
        jd_choice_75.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_75 = assortment_info_df_75['assortments']
choice_collection_75 = np.zeros((n,len(collection_75)))

frequency_collection_75 = np.zeros((n,len(collection_75)))

purchaseset_list_75 = jd_choice_75['purchaseset'].tolist()

inc_prod_num_75 = assortment_info_df_75['includ_prod_num']
choice_prob_75 = jd_choice_75['choice prob']
frequency_75 = jd_choice_75['purchasenum']

global_num = 0
for i in range(len(collection_75)):
    local_num =0
    while(local_num < inc_prod_num_75[i]):
        if purchaseset_list_75[global_num][0] in assortments_75[i]:
            choice_collection_75[purchaseset_list_75[global_num][0]][i] = choice_prob_75[global_num]
            frequency_collection_75[purchaseset_list_75[global_num][0]][i] = frequency_75[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_75[global_num])
            print(assortments_75[i])
            print('error!')
            break
#print(global_num)  

choice_check_75 = pd.DataFrame(choice_collection_75)
#choice_check_75.to_csv('instances/full_instances/full_75.csv')
frequency_collection_75 = frequency_collection_75.astype(int)
frequency_check_75 = pd.DataFrame(frequency_collection_75)
#frequency_check_75.to_csv('instances/full_instances/frequency_75.csv')

offer_times_75 = assortment_info_df_75['offer_times'].tolist()
print('the number of assortments with at least 75 purchases is',len(assortments_75))

the number of assortments with at least 75 purchases is 12


In [10]:
assortment_info_df_100 = assortment_info_df[(assortment_info_df['offer_times']>=100)]
assortments_100 = []
for i in range(len(assortment_info_df_100['assortments'])):
    assortments_100.append(assortment_info_df_100['assortments'][i])

jd_choice_100 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_100:
        jd_choice_100.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_100 = assortment_info_df_100['assortments']
choice_collection_100 = np.zeros((n,len(collection_100)))

frequency_collection_100 = np.zeros((n,len(collection_100)))

purchaseset_list_100 = jd_choice_100['purchaseset'].tolist()

inc_prod_num_100 = assortment_info_df_100['includ_prod_num']
choice_prob_100 = jd_choice_100['choice prob']
frequency_100 = jd_choice_100['purchasenum']

global_num=0
for i in range(len(collection_100)):
    local_num =0
    while(local_num < inc_prod_num_100[i]):
        if purchaseset_list_100[global_num][0] in assortments_100[i]:
            choice_collection_100[purchaseset_list_100[global_num][0]][i] = choice_prob_100[global_num]
            frequency_collection_100[purchaseset_list_100[global_num][0]][i] = frequency_100[global_num]
            global_num = global_num +1
            local_num = local_num +1

choice_check_100 = pd.DataFrame(choice_collection_100)
#choice_check_100.to_csv('instances/full_instances/full_100.csv')
frequency_collection_100 = frequency_collection_100.astype(int)
frequency_check_100 = pd.DataFrame(frequency_collection_100)
#frequency_check_100.to_csv('instances/full_instances/frequency_100.csv')

offer_times_100 = assortment_info_df_100['offer_times'].tolist()
print('the number of assortments with at least 100 purchases is',len(assortments_100))

the number of assortments with at least 100 purchases is 11


In [11]:
assortment_info_df_15 = assortment_info_df[(assortment_info_df['offer_times']>=15)]
assortments_15 = []
for i in range(len(assortment_info_df_15['assortments'])):
    assortments_15.append(assortment_info_df_15['assortments'][i])


jd_choice_15 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_15:
        jd_choice_15.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_15 = assortment_info_df_15['assortments']
choice_collection_15 = np.zeros((n,len(collection_15)))

frequency_collection_15 = np.zeros((n,len(collection_15)))

purchaseset_list_15 = jd_choice_15['purchaseset'].tolist()

inc_prod_num_15 = assortment_info_df_15['includ_prod_num']
choice_prob_15 = jd_choice_15['choice prob']
frequency_15 = jd_choice_15['purchasenum']

global_num = 0
for i in range(len(collection_15)):
    local_num =0
    while(local_num < inc_prod_num_15[i]):
        if purchaseset_list_15[global_num][0] in assortments_15[i]:
            choice_collection_15[purchaseset_list_15[global_num][0]][i] = choice_prob_15[global_num]
            frequency_collection_15[purchaseset_list_15[global_num][0]][i] = frequency_15[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_15[global_num])
            print(assortments_15[i])
            print('error!')
            break
#print(global_num)  

choice_check_15 = pd.DataFrame(choice_collection_15)
#choice_check_15.to_csv('instances/full_instances/full_15.csv')
frequency_collection_15 = frequency_collection_15.astype(int)
frequency_check_15 = pd.DataFrame(frequency_collection_15)
#frequency_check_15.to_csv('instances/full_instances/frequency_15.csv')

offer_times_15 = assortment_info_df_15['offer_times'].tolist()
print('the number of assortments with at least 15 purchases is',len(assortments_15))

the number of assortments with at least 15 purchases is 38


both MDM and RUM represent the data with approximately 99% confidence interval,  we set both z = 2.5758293 for both RUM and MDM

In [12]:
confidence_level = 0.99
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)

stardard_error_20 = np.zeros(frequency_collection_20.shape)
for i in range(stardard_error_20.shape[0]):
    for j in range(stardard_error_20.shape[1]):
        if frequency_collection_20[i][j]!=0:
            stardard_error_20[i][j] = np.sqrt((1-choice_collection_20[i][j])/frequency_collection_20[i][j])
            
lb_20 = np.zeros(frequency_collection_20.shape)
ub_20 = np.zeros(frequency_collection_20.shape)
for i in range(lb_20.shape[0]):
    for j in range(lb_20.shape[1]):
        if frequency_collection_20[i][j]!=0:
            lb_20[i][j] = choice_collection_20[i][j] * (1-target_z_score*stardard_error_20[i][j]) 
            ub_20[i][j] = choice_collection_20[i][j] * (1+target_z_score*stardard_error_20[i][j])

stardard_error_30 = np.zeros(frequency_collection_30.shape)
for i in range(stardard_error_30.shape[0]):
    for j in range(stardard_error_30.shape[1]):
        if frequency_collection_30[i][j]!=0:
            stardard_error_30[i][j] = np.sqrt((1-choice_collection_30[i][j])/frequency_collection_30[i][j])
            
lb_30 = np.zeros(frequency_collection_30.shape)
ub_30 = np.zeros(frequency_collection_30.shape)
for i in range(lb_30.shape[0]):
    for j in range(lb_30.shape[1]):
        if frequency_collection_30[i][j]!=0:
            lb_30[i][j] = choice_collection_30[i][j] * (1-target_z_score*stardard_error_30[i][j]) 
            ub_30[i][j] = choice_collection_30[i][j] * (1+target_z_score*stardard_error_30[i][j])
            
stardard_error_40 = np.zeros(frequency_collection_40.shape)
for i in range(stardard_error_40.shape[0]):
    for j in range(stardard_error_40.shape[1]):
        if frequency_collection_40[i][j]!=0:
            stardard_error_40[i][j] = np.sqrt((1-choice_collection_40[i][j])/frequency_collection_40[i][j])
            
lb_40 = np.zeros(frequency_collection_40.shape)
ub_40 = np.zeros(frequency_collection_40.shape)
for i in range(lb_40.shape[0]):
    for j in range(lb_40.shape[1]):
        if frequency_collection_40[i][j]!=0:
            lb_40[i][j] = choice_collection_40[i][j] * (1-target_z_score*stardard_error_40[i][j]) 
            ub_40[i][j] = choice_collection_40[i][j] * (1+target_z_score*stardard_error_40[i][j])

stardard_error_50 = np.zeros(frequency_collection_50.shape)
for i in range(stardard_error_50.shape[0]):
    for j in range(stardard_error_50.shape[1]):
        if frequency_collection_50[i][j]!=0:
            stardard_error_50[i][j] = np.sqrt((1-choice_collection_50[i][j])/frequency_collection_50[i][j])
            
lb_50 = np.zeros(frequency_collection_50.shape)
ub_50 = np.zeros(frequency_collection_50.shape)
for i in range(lb_50.shape[0]):
    for j in range(lb_50.shape[1]):
        if frequency_collection_50[i][j]!=0:
            lb_50[i][j] = choice_collection_50[i][j] * (1-target_z_score*stardard_error_50[i][j]) 
            ub_50[i][j] = choice_collection_50[i][j] * (1+target_z_score*stardard_error_50[i][j])
            
stardard_error_60 = np.zeros(frequency_collection_60.shape)
for i in range(stardard_error_60.shape[0]):
    for j in range(stardard_error_60.shape[1]):
        if frequency_collection_60[i][j]!=0:
            stardard_error_60[i][j] = np.sqrt((1-choice_collection_60[i][j])/frequency_collection_60[i][j])
            
lb_60 = np.zeros(frequency_collection_60.shape)
ub_60 = np.zeros(frequency_collection_60.shape)
for i in range(lb_60.shape[0]):
    for j in range(lb_60.shape[1]):
        if frequency_collection_60[i][j]!=0:
            lb_60[i][j] = choice_collection_60[i][j] * (1-target_z_score*stardard_error_60[i][j]) 
            ub_60[i][j] = choice_collection_60[i][j] * (1+target_z_score*stardard_error_60[i][j])            


### full data with different offertimes

In [13]:
pred_offer_times_list = [20,30,40,50,60]
pred_full_instance  = [choice_collection_20,choice_collection_30,choice_collection_40,choice_collection_50,choice_collection_60]
pred_full_lb = [lb_20,lb_30,lb_40,lb_50,lb_60]
pred_full_ub = [ub_20,ub_30,ub_40,ub_50,ub_60]

pred_full_collection = [collection_20,collection_30,collection_40,collection_50,collection_60]
pred_full_collection_size = [len(offer_times_20),len(offer_times_30),len(offer_times_40),len(offer_times_50),len(offer_times_60)]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
#pred_instance_size = [100,100,100,50,50]
pred_instance_size = [50,50,50,50,50]

price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

pred_full_offertimes = [offer_times_20,offer_times_30,offer_times_40,offer_times_50,offer_times_60]

train-test split

In [14]:
all_train_instances = []
all_train_collection = []
all_test_instance = []
all_test_collection = []
all_train_lb = []
all_train_ub = []
all_train_offertimes = []

for i in range(len(pred_instance_size)):
    train_instances_collection = []
    train_collection = []
    train_lb_collection = []
    train_ub_collection = []
    train_offertimes_collection = []
    
    test_instance_collection = []
    test_collection = []
    
    test_index_collection = []
    
    
    for j in range(pred_instance_size[i]):
        
        curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
        curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
        curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
        
        curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
        
        curr_train_collection = []
        curr_test_collection = []
        curr_offertimes_lst = []
        
        while(len(curr_train_collection)==0):
            
            chosen_test_idx = random.sample(range(len(pred_full_collection[i])),pred_test_collection_size[i])
            chosen_test_idx.sort()
            
            chosen_train_idx = [x for x in range(len(pred_full_collection[i])) if x not in chosen_test_idx ]
    
            for k in range(len(chosen_train_idx)):
                curr_train_collection.append(pred_full_collection[i][chosen_train_idx[k]])
                curr_train_instance[:,k] = pred_full_instance[i][:,chosen_train_idx[k]]
                curr_train_lb[:,k] = pred_full_lb[i][:,chosen_train_idx[k]]
                curr_train_ub[:,k] = pred_full_ub[i][:,chosen_train_idx[k]]
                
                curr_offertimes_lst.append(pred_full_offertimes[i][chosen_train_idx[k]])
                
            for k in range(len(chosen_test_idx)):
                curr_test_collection.append(pred_full_collection[i][chosen_test_idx[k]])
                curr_test_instance[:,k] = pred_full_instance[i][:,chosen_test_idx[k]]
                
            ## check if the training instance includes all the testing products in the test instance
            # ensure each test product has been offered at least  once
            new_lst_train = sum(curr_train_collection,[])
            unique_numbers_train = list(set(new_lst_train)) 
            
            new_lst_test = sum(curr_test_collection,[])
            unique_numbers_test = list(set(new_lst_test)) 
            
            for num in unique_numbers_test:
                if num not in unique_numbers_train:
                    #print('the unique train numbers',unique_numbers_train)
                    #print('the unique test numbers',unique_numbers_test)
                    curr_train_collection = []
                    curr_test_collection = []
                    curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                    curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                    curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                    curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                    curr_offertimes_lst = []
            if chosen_test_idx in test_index_collection:
                #print('check if exisits repeated indexes ')
                curr_train_collection = []
                curr_test_collection = []
                curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                curr_offertimes_lst = []
            elif len(curr_test_collection)>0:
                test_index_collection.append(chosen_test_idx)
        #print('chosen training indexes',chosen_train_idx)
        #print('chosen testing indexes',chosen_test_idx)
        #print('curr train collection\n',curr_train_collection)
        #print('curr test collection\n',curr_test_collection)
        ''' df_train = pd.DataFrame(curr_train_instance)
        df_test = pd.DataFrame(curr_test_instance)
        df_offertimes = pd.DataFrame(curr_offertimes_lst)
        #print('curr train instance\n',df_train)
        #print('curr test instance\n',df_test)
        
        ## output the training and testing instance
        df_train.to_csv('instances/train_instances/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_test.to_csv('instances/test_instances/test_'+str(pred_offer_times_list[i])+'/test_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_offertimes.to_csv('instances/train_offertimes/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
         '''
        train_instances_collection.append(curr_train_instance)
        train_collection.append(curr_train_collection)
        train_lb_collection.append(curr_train_lb)
        train_ub_collection.append(curr_train_ub)
        train_offertimes_collection.append(curr_offertimes_lst)
                
        test_instance_collection.append(curr_test_instance)
        test_collection.append(curr_test_collection)
        
    all_train_instances.append(train_instances_collection)
    all_train_collection.append(train_collection)
    all_train_lb.append(train_lb_collection)
    all_train_ub.append(train_ub_collection)
    all_train_offertimes.append(train_offertimes_collection)
    all_test_instance.append(test_instance_collection)
    all_test_collection.append(test_collection)    
        

In [15]:
pd.DataFrame(all_train_instances[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.735746,0.705128,0.647154,0.773333,0.738497,0.741758,0.736842,0.745098,0.847826,0.846774,...,0.75,0.692308,0.715447,0.791667,0.78125,0.822222,0.739130,0.642857,0.714286,0.688805
1,0.264254,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.176471,0.130435,0.048387,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.294872,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.352846,0.000000,0.000000,0.000000,0.192982,0.000000,0.000000,0.000000,...,0.25,0.269231,0.211382,0.166667,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.226667,0.000000,0.000000,0.000000,0.078431,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.18750,0.022222,0.043478,0.035714,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,...,0.00,0.038462,0.000000,0.000000,0.03125,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.261503,0.000000,0.000000,0.000000,0.000000,0.104839,...,0.00,0.000000,0.073171,0.000000,0.00000,0.155556,0.000000,0.000000,0.142857,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.258242,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.217391,0.000000,0.142857,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.041667,0.00000,0.000000,0.000000,0.285714,0.000000,0.311195


In [16]:
pd.DataFrame(all_test_instance[0][0])

,0,1,2,3,4
0,0.91442,0.625,0.686567,0.748744,0.757576
1,0.00000,0.000,0.000000,0.000000,0.000000
2,0.00000,0.000,0.000000,0.000000,0.000000
3,0.00000,0.275,0.000000,0.000000,0.000000
4,0.00000,0.000,0.000000,0.000000,0.000000
5,0.08558,0.000,0.044776,0.060302,0.000000
6,0.00000,0.000,0.268657,0.000000,0.121212
7,0.00000,0.100,0.000000,0.000000,0.000000
8,0.00000,0.000,0.000000,0.190955,0.121212


In [17]:
pd.DataFrame(all_train_lb[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.698136,0.572138,0.612058,0.722495,0.711180,0.693512,0.586606,0.587908,0.711411,0.763453,...,0.573645,0.459156,0.610654,0.578135,0.593010,0.675416,0.503286,0.409610,0.517594,0.636856
1,0.226645,0.000000,0.000000,0.000000,0.000000,0.000000,-0.016976,0.038969,0.002530,-0.001249,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.161881,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.317749,0.000000,0.000000,0.000000,0.058341,0.000000,0.000000,0.000000,...,0.073645,0.045161,0.116555,-0.029283,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.175829,0.000000,0.000000,0.000000,-0.018539,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.009773,-0.034379,-0.066053,-0.054622,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.033645,0.000000,...,0.000000,-0.058685,0.000000,0.000000,-0.047977,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.234185,0.000000,0.000000,0.000000,0.000000,0.033976,...,0.000000,0.000000,0.012688,0.000000,0.000000,0.016388,0.000000,0.000000,-0.009499,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.209995,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.004146,0.000000,-0.009499,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.063400,0.000000,0.000000,0.000000,0.065807,0.000000,0.259247


In [18]:
pd.DataFrame(all_train_ub[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.773355,0.838119,0.682251,0.824171,0.765815,0.790005,0.887078,0.902288,0.984241,0.930095,...,0.926355,0.925459,0.820241,1.005198,0.969490,0.969028,0.974975,0.876104,0.910977,0.740753
1,0.301864,0.000000,0.000000,0.000000,0.000000,0.000000,0.157327,0.313972,0.258339,0.098024,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.427862,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.387942,0.000000,0.000000,0.000000,0.327624,0.000000,0.000000,0.000000,...,0.426355,0.493300,0.306209,0.362617,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.277505,0.000000,0.000000,0.000000,0.175402,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.365227,0.078823,0.153009,0.126051,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077123,0.000000,...,0.000000,0.135608,0.000000,0.000000,0.110477,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.288820,0.000000,0.000000,0.000000,0.000000,0.175701,...,0.000000,0.000000,0.133654,0.000000,0.000000,0.294723,0.000000,0.000000,0.295214,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.306488,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.438928,0.000000,0.295214,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.146733,0.000000,0.000000,0.000000,0.505622,0.000000,0.363144


In [19]:
all_test_collection[0][0]

[[0, 5], [0, 3, 7], [0, 5, 6], [0, 5, 8], [0, 6, 8]]

In [20]:
## RUM limit formulation
def find_element_permutation_indexes(n, partial_lists):
    perm_collection = []
    
    elements = list(range(n))
    perms = list(permutations(elements))
    #index_permutations = {perm: idx for idx, perm in enumerate(perms)}
    result = {}

    for partial_list in partial_lists:
        partial_result = {}
        perm_element = []
        for element in partial_list:
            element_indexes = [idx for idx, perm in enumerate(perms) if perm.index(element) == min(perm.index(e) for e in partial_list)]
            partial_result[element] = element_indexes
            perm_element.append(element_indexes)
        result[tuple(partial_list)] = partial_result
        perm_collection.append(perm_element)
    return result,perm_collection

def find_product_notin_assortment(collection,n):
    notin_collection = []
    for i in range(len(collection)):
        notin_assortment = []
        for j in range(n):
            if j not in collection[i]:
                notin_assortment.append(j)
        notin_collection.append(notin_assortment)
    return notin_collection


In [21]:
def rum_feasiblity(data,collection,perm_collection):
    n = data.shape[0]
    
    fact = math.factorial(n)
    
    # the RUM feasibility LP
    model = gp.Model('rum_feasibility')
    model.setParam('OutputFlag', 0)
    lam = model.addVars(fact,lb=0.0, vtype=gp.GRB.CONTINUOUS, name='lam')
    
    #model.addConstrs(lam[i]>=0 for i in range(fact))
    model.addConstr(sum(lam[i] for i in range(fact))==1)
    
    # caculate RUM choice probability for each product in the assortment
    for i in range(len(collection)):
        for j in range(len(collection[i])):
            model.addConstr(sum(lam[k] for k in perm_collection[i][j])- data[collection[i][j]][i]==0)
    
    model.setObjective(0)
    #model.write("model.lp")
    model.optimize()
    # Access the optimal solution

    if model.Status == 2: #optimal
        return [1,model.Runtime]
        #return [1, used_time]
    
    if model.Status == 3: #infeasible
        return [0,model.Runtime]


In [22]:
def robust_rum_prediction(data,collection,perm_collection,unseen_assortment,unseen_assortment_perm,price):
    start_time = time.time()
    
    n = data.shape[0]
    
    fact = math.factorial(n)
    
    model = gp.Model('rum')
    #model.Params.FeasibilityTol = 1e-5
    model.setParam('OutputFlag', 0)
    lam = model.addVars(fact,lb=0.0, vtype=gp.GRB.CONTINUOUS, name='lam')
    x = model.addVars(len(unseen_assortment),lb=0.0,vtype=gp.GRB.CONTINUOUS,name='x')
    
    #model.addConstrs(lam[i]>=0 for i in range(fact))
    model.addConstr(sum(lam[i] for i in range(fact))==1)
    #model.addConstr(sum(x[i] for i in range(len(unseen_assortment))) == 1)
    
    # caculate RUM choice probability for each product in the assortment
    for i in range(len(collection)):
        for j in range(len(collection[i])):
            model.addConstr(sum(lam[k] for k in perm_collection[i][j])- data[collection[i][j]][i]==0)
    
    # calculate RUM choice probability for the unseen assortment
    for i in range(len(unseen_assortment)):
        model.addConstr(sum(lam[k] for k in unseen_assortment_perm[i]) - x[i] == 0)
    
    # minimize the revenue within all possible rums
    model.setObjective(gp.quicksum(price[unseen_assortment[i]]*x[i] for i in range(len(unseen_assortment))),sense=gp.GRB.MINIMIZE)
   
    model.optimize()
    used_time = time.time() - start_time
    #status = model.status
    #print("Gurobi optimization status:", status)
    
    # Check if the optimization was successful
    if model.status == gp.GRB.OPTIMAL:
        # Access the objective value
        #obj_value = model.objVal
        #print(f"Optimal Objective Value: {obj_value}")  
        #print(f'current testing assortment',unseen_assortment)
        return [model.objVal,[x[i].x for i in range(len(unseen_assortment))], model.Runtime,used_time]
    if model.status == gp.GRB.INFEASIBLE:
        print("Optimization was not successful.")
        #print(f'current testing assortment',unseen_assortment)
        # Compute the Infeasibility Proof (IIS)
        ''' model.computeIIS()
        infeas_constraints = [c.constrName for c in model.getConstrs() if c.IISConstr]
        print("The following constraints contribute to infeasibility:")
        print(infeas_constraints) '''
        model.write("rumPre_infeas_model_1e-6.lp")
        return [-1, [ 0 for _ in range(len(unseen_assortment))],model.Runtime,used_time]
    
    

In [23]:
def robust_rum_prediction_tol(data,collection,perm_collection,unseen_assortment,unseen_assortment_perm,price,tol):
    start_time = time.time()
    n = data.shape[0]
    
    fact = math.factorial(n)
    
    model = gp.Model('rum')
    model.Params.FeasibilityTol = tol
    model.setParam('OutputFlag', 0)
    lam = model.addVars(fact,lb=0.0, vtype=gp.GRB.CONTINUOUS, name='lam')
    x = model.addVars(len(unseen_assortment),lb=0.0,vtype=gp.GRB.CONTINUOUS,name='x')
    
    #model.addConstrs(lam[i]>=0 for i in range(fact))
    model.addConstr(sum(lam[i] for i in range(fact))==1)
    #model.addConstr(sum(x[i] for i in range(len(unseen_assortment))) == 1)
    
    # caculate RUM choice probability for each product in the assortment
    for i in range(len(collection)):
        for j in range(len(collection[i])):
            model.addConstr(sum(lam[k] for k in perm_collection[i][j])- data[collection[i][j]][i]==0)
    
    # calculate RUM choice probability for the unseen assortment
    for i in range(len(unseen_assortment)):
        model.addConstr(sum(lam[k] for k in unseen_assortment_perm[i]) - x[i] == 0)
    
    # minimize the revenue within all possible rums
    model.setObjective(gp.quicksum(price[unseen_assortment[i]]*x[i] for i in range(len(unseen_assortment))),sense=gp.GRB.MINIMIZE)
   
    model.optimize()
    #status = model.status
    #print("Gurobi optimization status:", status)
    used_time= time.time() - start_time
    # Check if the optimization was successful
    if model.status == gp.GRB.OPTIMAL:
        # Access the objective value
        #obj_value = model.objVal
        #print(f"Optimal Objective Value: {obj_value}")  
        #print(f'current testing assortment',unseen_assortment)
        return [model.objVal,[x[i].x for i in range(len(unseen_assortment))], model.Runtime,used_time]
    if model.status == gp.GRB.INFEASIBLE:
        print("Optimization was not successful.")
        #print(f'current testing assortment',unseen_assortment)
        # Compute the Infeasibility Proof (IIS)
        ''' model.computeIIS()
        infeas_constraints = [c.constrName for c in model.getConstrs() if c.IISConstr]
        print("The following constraints contribute to infeasibility:")
        print(infeas_constraints) '''
        #model.write("rumPre_infeas_model_tol.lp")
        return [-1, [ 0 for _ in range(len(unseen_assortment))],model.Runtime,used_time]
    
    #return [-2, [ 0 for _ in range(len(unseen_assortment))],model.Runtime]
    
    

In [24]:
## prediction for the current testing collection of assortments 
def rum_prediction(curr_test_instance,curr_test_collection,price,curr_train_instance,curr_train_collection,curr_train_perm_collection,curr_test_perm_collection):
    ''' true assortment ranking '''

    true_revenue = []

    lb_rum_revenue = [] 

    lb_rum_runtime = []
    actual_run_runtime = []
    
    for k in range(len(curr_test_collection)):
        
        unseen_assortment = curr_test_collection[k]
        unseen_perm_collection = curr_test_perm_collection[k]
        # true revenue and true choice probabilities for current testing assortment
        curr_revenue = np.dot(curr_test_instance[:,k],price)
        true_revenue.append(curr_revenue)
        #true_choice_prob.append(curr_test_instance[:,k])
        
        # be careful for the output sequence
        
        curr_rum_lb_result = robust_rum_prediction(curr_train_instance,curr_train_collection,curr_train_perm_collection,unseen_assortment,unseen_perm_collection,price)
        if curr_rum_lb_result[0] == -1:
            tol = 1e-5
            curr_rum_lb_result = robust_rum_prediction_tol(curr_train_instance,curr_train_collection,curr_train_perm_collection,unseen_assortment,unseen_perm_collection,price,tol)
            while(curr_rum_lb_result[0] == -1 and tol <1e-2):
                print('current prediction needs a significant tolerence')
                tol = tol*10
                curr_rum_lb_result = robust_rum_prediction_tol(curr_train_instance,curr_train_collection,curr_train_perm_collection,unseen_assortment,unseen_perm_collection,price,tol)
            
        lb_rum_revenue.append(curr_rum_lb_result[0])
        
        ## record the runtime result
        lb_rum_runtime.append(curr_rum_lb_result[2])
        actual_run_runtime.append(curr_rum_lb_result[-1])
        print(f'robust RUM revenue prediction of {unseen_assortment} is {curr_rum_lb_result[0]} with actual runtime {curr_rum_lb_result[3]}')
       
    return true_revenue,lb_rum_revenue,lb_rum_runtime,actual_run_runtime

In [25]:
def robust_rum_prediction_w_ci(lb,ub,data,collection,perm_collection,produts_notin_collection,unseen_assortment,unseen_assortment_perm,price):
    start_time = time.time()
    n,m = data.shape # number of products
    fact = math.factorial(n) # number of permutations
    
    model = gp.Model('rum_ci') #define a model
    model.setParam('OutputFlag', 0)
    
    # define nonnegative continuous variables for lambda, the probability of each permutation
    lam = model.addVars(fact,lb=0.0, vtype=gp.GRB.CONTINUOUS, name='lam') 
    # define nonnegative continuous variables for choice probability of each product in each train assortment
    x = model.addVars(n, m, vtype=gp.GRB.CONTINUOUS, lb=0.0, name="x") 
    # define nonnegative continuous variables for choice probability of each product in the test assortment 
    y = model.addVars(len(unseen_assortment),lb=0.0,vtype=gp.GRB.CONTINUOUS,name='y')
    
    #normalization constraint of lambda
    model.addConstr(sum(lam[i] for i in range(fact))==1,name="Normalization")
    
    # caculate RUM choice probability for each product in the train assortment
    for i in range(len(collection)):
        # normalization constraint for each assortment. 
        #model.addConstr(gp.quicksum(x[k, i] for k in collection[i]) == 1)
        for j in range(len(collection[i])):
            # constraints for RUM choice probabilities 
            model.addConstr(sum(lam[k] for k in perm_collection[i][j])- x[collection[i][j],i]==0,name=f"prob_product{collection[i][j]}_assortment_{i}")
            
            # constraints for the lower bound and upper bound values
            model.addConstr(x[collection[i][j],i]>= lb[collection[i][j],i],name=f"lb_product{collection[i][j]}_assortment_{i}")
            model.addConstr(x[collection[i][j],i]<= ub[collection[i][j],i],name=f"ub_product{collection[i][j]}_assortment_{i}")
        
        for k in produts_notin_collection[i]:
            # constraints for products that are not in the assortments
            model.addConstr(x[k,i] == 0,name=f"prob_product{k}_assortment_{i}")
    
    # calculate RUM choice probability for the unseen assortment
    for i in range(len(unseen_assortment)):
        model.addConstr(sum(lam[k] for k in unseen_assortment_perm[i]) - y[i] == 0,name=f'unseen_prob_product{unseen_assortment[i]}')
    
    # Objective
    obj = gp.quicksum(price[unseen_assortment[i]] * y[i] for i in range(len(unseen_assortment)))
    model.setObjective(obj, GRB.MINIMIZE)
    #model.setObjective(0)
    model.optimize()
    used_time = time.time() - start_time
    #status = model.status
    #print("Gurobi optimization status:", status)
    # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        print(f"RUM robust prediction revenue of assortment {unseen_assortment} is : {model.objVal} with runtime {used_time}")  
        #model.write("rum_ci.sol")
        #model.write("rum_ci.lp")
        x_values = [[x[i, j].X for j in range(m)] for i in range(n)]
        x_values = np.array(x_values)
        #print(f'current testing assortment',unseen_assortment)
        return [model.objVal,[y[i].x for i in range(len(unseen_assortment))], model.Runtime,used_time]
    else:
        print('RUM CI: No solution found or an error occurred.')
        return [-1]
    

In [26]:
def kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    num = len(values1)
    assert len(values2) == num, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(num), np.arange(num))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered/2 

In [27]:
## record the kendall tau distance between the predicted ranking of rum and the truth
all_rum_lb_distance_list = []  

## record the true revenue of the predicted best assortment
all_best_true_revenue_list = []
all_best_revenue_rum_lb = []  

## record the difference in best assortment
all_rum_lb_best_rev_diff = []

## record all the true and predicted revenues
all_true_revenue_list = []
all_rum_lb_revenue_list = []

## choice probability under the revenue prediction 
''' all_true_choice_prob_list = [] 
all_rum_lb_rev_prob_list = []

all_rum_lb_sales_list = [] '''

all_true_ranking = []
all_rum_lb_ranking = []

## record the runtime of rum prediction
all_rum_rev_lb_runtime = []
all_rum_actual_runtime = []

for i in range(len(pred_full_collection)):
    
    
    # define as a container for quantity of the same collection size 
    # The following lists are of the same size as the instance size
    
    true_best_rev_collection = [] # a container for groundtruth best reveue 
    rum_lb_best_rev_collection = [] # a container for the best revenue predicted by robust rum
    
    true_ranking_collection = [] # a container for groundtruth reveue ranking 
    rum_lb_ranking_collection = [] # a container for ranking by robust rum
    
    rum_lb_distance_collection = [] # a container for kendall tau distance between true ranking and the ranking by robust rum
   
    ''' true_choice_prob_collection = []
    rum_lb_rev_prob_collection = []
   
    rum_lb_sales_frac_collection = [] '''

    true_revenue_collection = []
    rum_lb_revenue_collection = []
   
    rum_lb_runtime_collection = []
    rum_actual_runtime_collection = []
    
    
    for j in range(pred_instance_size[i]):
        print(f'testing for offertimes {pred_offer_times_list[i]}, {j}th instance')
        # the following lists are of the same size as number of testing assortments
        ''' true assortment ranking '''
        #true_choice_prob = [] 
        true_revenue = []
        
        ''' lb_rum_rev_prob = []
        lb_rum_sales = [] '''
        lb_rum_revenue = [] 
        lb_rum_runtime = []
        actual_runtime = [] 
        
        produts_notin_collection = find_product_notin_assortment(all_train_collection[i][j],n)
        train_element_indexes,train_perm_collection = find_element_permutation_indexes(n, all_train_collection[i][j])
        test_element_indexes, test_perm_collection = find_element_permutation_indexes(n, all_test_collection[i][j])
        
 
        for k in range(len(all_test_collection[i][j])):
            
            unseen_assortment = all_test_collection[i][j][k]
            
            # true revenue and true choice probabilities for current testing assortment
            curr_revenue = np.dot(all_test_instance[i][j][:,k],price)
            true_revenue.append(curr_revenue)
            #true_choice_prob.append(all_test_instance[i][j][:,k])
            
            # rum revenue prediction
            # robust_rum_prediction_w_ci(lb,ub,data,collection,perm_collection,produts_notin_collection,unseen_assortment,unseen_assortment_perm,price)
            curr_rum_lb_result = robust_rum_prediction_w_ci(all_train_lb[i][j],all_train_ub[i][j],all_train_instances[i][j],all_train_collection[i][j],train_perm_collection,produts_notin_collection,unseen_assortment,test_perm_collection[k],price)
        
            ## record the revenue result
            lb_rum_revenue.append(curr_rum_lb_result[0])
        
            ''' ## record the choice probability result
            lb_prob = np.zeros(n)
            lb_prob_result = curr_rum_lb_result[1]
            
            for l in range(len(unseen_assortment)):
                lb_prob[unseen_assortment[l]] = lb_prob_result[l]
        
            lb_rum_rev_prob.append(lb_prob) '''
    
            ## record the runtime result
            lb_rum_runtime.append(curr_rum_lb_result[2])
            actual_runtime.append(curr_rum_lb_result[-1])
            
            
            ''' ## rum choice probability interval prediction
            lb_sales_frac = np.zeros(n)

            for l in range(len(unseen_assortment)):
                unit_price = np.zeros(n)
                unit_price[unseen_assortment[l]] = 1
                ## unit price is a vector that takes value 1 only at unseen_assortment[l] and all zeros for other n-1 elements
                lb_sales_frac[unseen_assortment[l]] = robust_rum_prediction_w_ci(all_train_lb[i][j],all_train_ub[i][j],all_train_instances[i][j],all_train_collection[i][j],train_perm_collection,produts_notin_collection,unseen_assortment,test_perm_collection[k],unit_price)[0]
            
            lb_rum_sales.append(lb_sales_frac) '''
                
        true_revenue_collection.append(true_revenue)
        #true_choice_prob_collection.append(true_choice_prob)
        
        rum_lb_revenue_collection.append(lb_rum_revenue)   
        #rum_lb_rev_prob_collection.append(lb_rum_rev_prob)
        
        rum_lb_runtime_collection.append(lb_rum_runtime)
        rum_actual_runtime_collection.append(actual_runtime)
        ## sales fraction record
        #rum_lb_sales_frac_collection.append(lb_rum_sales)
        
        # true assortment ranking for test instance [i][j]
        curr_true_rank = np.argsort(-np.array(true_revenue))  
        true_ranking_collection.append(curr_true_rank)
        # true best assortment revenue for test instance [i][j]
        true_best_rev_collection.append(true_revenue[curr_true_rank[0]])
        
        # lb rum assortment ranking for test instance [i][j] 
        curr_rum_lb_rank  = np.argsort(-np.array(lb_rum_revenue)) 
        rum_lb_ranking_collection.append(curr_rum_lb_rank)
        rum_lb_best_rev_collection.append(true_revenue[curr_rum_lb_rank[0]])
        
        #  rum kendallTau Distance for current instance [i][j]
        curr_rum_lb_dist = kendall_tau_distance(curr_true_rank, curr_rum_lb_rank)
        rum_lb_distance_collection.append(curr_rum_lb_dist)
    
    # for each test instance [i][j] kendall tau distance is a number
    all_rum_lb_distance_list.append(rum_lb_distance_collection)
    
    # for each test instance [i][j] best revenue is a number
    all_best_true_revenue_list.append(true_best_rev_collection) 
    all_best_revenue_rum_lb.append(rum_lb_best_rev_collection)
    
    # for each test instance [i][j] best revenue difference is a number
    all_rum_lb_best_rev_diff.append((np.array(true_best_rev_collection) - np.array(rum_lb_best_rev_collection))/ np.array(true_best_rev_collection))
    
    # for each test instance [i][j], true revenue is a list of the same size as the number of testing assortments
    all_true_revenue_list.append(true_revenue_collection)
    #all_true_choice_prob_list.append(true_choice_prob_collection)
    all_rum_lb_revenue_list.append(rum_lb_revenue_collection)
    #all_rum_lb_rev_prob_list.append(rum_lb_rev_prob_collection)
    
    #all_rum_lb_sales_list.append(rum_lb_sales_frac_collection)
    
    # for each test instance [i][j], true ranking is a list of the same size as the number of testing assortments
    all_true_ranking.append(true_ranking_collection)
    all_rum_lb_ranking.append(rum_lb_ranking_collection)

    ## for each test instance [i][j], runtime is a list of the same size as the number of testing assortments
    all_rum_rev_lb_runtime.append(rum_lb_runtime_collection)
    all_rum_actual_runtime.append(rum_actual_runtime_collection)
    

testing for offertimes 20, 0th instance
Academic license - for non-commercial use only - expires 2024-12-10
Using license file /Users/autumn/gurobi.lic
RUM robust prediction revenue of assortment [0, 5] is : 0.0 with runtime 113.70269894599915
RUM robust prediction revenue of assortment [0, 3, 7] is : 0.12423996484134239 with runtime 103.63564014434814
RUM robust prediction revenue of assortment [0, 5, 6] is : 0.11100364345591643 with runtime 107.58276128768921
RUM robust prediction revenue of assortment [0, 5, 8] is : 0.3128180291590198 with runtime 104.0761547088623
RUM robust prediction revenue of assortment [0, 6, 8] is : 0.14667187565844222 with runtime 102.99883985519409
testing for offertimes 20, 1th instance
RUM robust prediction revenue of assortment [0, 3] is : 0.04557305442625592 with runtime 115.51846194267273
RUM robust prediction revenue of assortment [0, 4] is : 0.016193182129382398 with runtime 99.66443490982056
RUM robust prediction revenue of assortment [0, 1, 4] is :

In [28]:
## adding details 1
for i in range(len(pred_offer_times_list)):
    df_rum_details1 = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'rum_lb_dist':all_rum_lb_distance_list[i],
                                    'true_best_rev':all_best_true_revenue_list[i],'rum_lb_best_rev':all_best_revenue_rum_lb[i],
                                    'rum_lb_best_rev_diff':all_rum_lb_best_rev_diff[i]})
    df_rum_details1.to_csv('prediction/rum/revenue_prediction/details1/'+str(pred_offer_times_list[i])+'.csv')

In [29]:
## adding details 2 about ranking, revenue, choice probability
for i in range(len(pred_offer_times_list)):
    for j in range(50):
        
        true_rank = all_true_ranking[i][j]
        lb_rank = all_rum_lb_ranking[i][j]
        
        df_rank = pd.DataFrame({'true_rank':true_rank,'lb_rank':lb_rank})
        df_rank.to_csv('prediction/rum/revenue_prediction/details2/ranking/offertimes'+str(pred_offer_times_list[i])+'/rank_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        true_rev = all_true_revenue_list[i][j]
        lb_rev = all_rum_lb_revenue_list[i][j]
        
        df_rev = pd.DataFrame({'true_rev':true_rev,'lb_rev':lb_rev})
        df_rev.to_csv('prediction/rum/revenue_prediction/details2/revenue/offertimes'+str(pred_offer_times_list[i])+'/rev_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        ''' true_prob = all_true_choice_prob_list[i][j]
        lb_prob = all_rum_lb_rev_prob_list[i][j]
        
        
        df_true_prob = pd.DataFrame(true_prob).T
        df_lb_prob = pd.DataFrame(lb_prob).T
       
        
        df_true_prob.to_csv('prediction/rum/revenue_prediction/details2/probabilities/offertimes'+str(pred_offer_times_list[i])+'/true/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_lb_prob.to_csv('prediction/rum/revenue_prediction/details2/probabilities/offertimes'+str(pred_offer_times_list[i])+'/rum_lb/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        '''
        lb_runtime = all_rum_rev_lb_runtime[i][j]
       
        df_runtime = pd.DataFrame({'lb_runtime':lb_runtime})
        df_runtime.to_csv('prediction/rum/revenue_prediction/details2/runtime/offertimes'+str(pred_offer_times_list[i])+'/runtime_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        actual_runtime = all_rum_actual_runtime[i][j]
        df_actual_runtime = pd.DataFrame({'rum_runtime':actual_runtime})
        df_actual_runtime.to_csv('prediction/rum/revenue_prediction/details2/actual_runtime/offertimes'+str(pred_offer_times_list[i])+'/runtime_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        ''' ## using sales fraction prediction
        ## record the choice probability prediction interval
        lb_sales = all_rum_lb_sales_list[i][j]
     
        df_lb_sales = pd.DataFrame(lb_sales).T
      
        
        df_true_prob.to_csv('prediction/rum/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/true/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_lb_sales.to_csv('prediction/rum/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/rum_lb/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
         '''

In [30]:
## adding details 2 about runtime 
all_lb_avg_runtime = []

for i in range(len(pred_offer_times_list)):
    
    avg_lb_runtime_collection = []

    for j in range(50):
        avg_lb_runtime_collection.append(np.mean(np.array(all_rum_rev_lb_runtime[i][j])))
   
    all_lb_avg_runtime.append(np.mean(np.array(avg_lb_runtime_collection)))


In [31]:
## adding details 2 about runtime 
all_actual_avg_runtime = []

for i in range(len(pred_offer_times_list)):
    
    avg_actual_runtime_collection = []

    for j in range(pred_instance_size[i]):
        avg_actual_runtime_collection.append(np.mean(np.array(all_rum_actual_runtime[i][j])))
   
    all_actual_avg_runtime.append(np.mean(np.array(avg_actual_runtime_collection)))

In [32]:
avg_rum_lb_dist = []

avg_rum_lb_best_rev_diff = []

for i in range(len(pred_offer_times_list)):
    avg_rum_lb_dist.append(np.mean(all_rum_lb_distance_list[i]))
 
    avg_rum_lb_best_rev_diff.append(np.mean(all_rum_lb_best_rev_diff[i]))

df_rum_jd_summary = pd.DataFrame({'offertimes':pred_offer_times_list,'train_size':pred_train_collection_size,'test_size':pred_test_collection_size,
                                   'avg_lb_dist':avg_rum_lb_dist,
                                   'avg_lb_best_rev_diff':avg_rum_lb_best_rev_diff,
                                   'avg_lb_runtime':all_lb_avg_runtime,
                                   'rum_actual_runtime':all_actual_avg_runtime})
df_rum_jd_summary.to_csv('prediction/rum/jd_rum_summary.csv')

In [33]:
df_rum_jd_summary

,offertimes,train_size,test_size,avg_lb_dist,avg_lb_best_rev_diff,avg_lb_runtime,rum_actual_runtime
0,20,24,5,3.16,0.111450,3.097602,152.814778
1,30,20,4,2.44,0.163216,2.461284,128.550709
2,40,16,3,1.34,0.172062,2.225715,126.695435
3,50,12,3,1.00,0.154974,1.207443,54.950283
4,60,11,2,0.34,0.089010,1.136671,52.121302
